In [5]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

In [6]:
from dask.distributed import Client
client = Client()
client

/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52685 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52685/status,
Dashboard: http://127.0.0.1:52685/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52686,Workers: 5
Dashboard: http://127.0.0.1:52685/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:52704,Total threads: 2
Dashboard: http://127.0.0.1:52709/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:52691,


In [7]:
print("Reading jsonl file...")

block_size = blocksize = 2**20 * 50  # 400MB
# number_rows_to_read = 200000


import pyarrow as pa
test_file = "otto-recsys-test.jsonl"

event_schema = pa.struct([("aid", pa.int64()), ("ts", pa.int64()), ("type", pa.string())])
schema = pa.schema(
    [
        ("session", pa.int64()),
        (
            "events",
            pa.list_(event_schema),
        ),
    ]
)

df = dd.read_json(
    f"archive/{test_file}",
    lines=True,
    blocksize=block_size,
)

# parquet_file = "archive/parquet/"


# name_function = lambda x: f"otto-recsys-train-{x}.parquet"
# print("Writing parquet file...")
# df.to_parquet(parquet_file, schema=schema,name_function=name_function)
# print("Done!")

Reading jsonl file...


In [8]:
mapper = lambda x: pd.json_normalize(
    x.to_dict(orient="records"),
    record_path=["events"],
    meta=["session"],
    errors="ignore",
)

meta = {
    "aid": np.int64,
    "ts": np.int64,
    "type": np.str,
    "session": np.int64,
}

print("Normalizing jsonl file...")
normalized = df.map_partitions(mapper, meta=meta)

print("Writing normalized jsonl file to parquet...")
normalized.to_parquet(f"archive/{test_file}-parquet")

Normalizing jsonl file...
Writing normalized jsonl file to parquet...


2023-09-27 17:35:33,084 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2023-09-27 17:35:41,672 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2023-09-27 17:35:43,435 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2023-09-27 17:35:44,500 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-09-27 17:35:45,279 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-09-27 17:35:50,793 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2023-09-27 17:35:55,838 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-09-27 17:35:56,644 - distributed.utils_perf - WARNING - full garbage collections took

In [10]:
df = dd.read_parquet(f"archive/{test_file}-parquet")
df.compute()

,aid,ts,type,session
0,1517085,1659304800025,clicks,0
1,1563459,1659304904511,clicks,0
2,1309446,1659367439426,clicks,0
3,16246,1659367719997,clicks,0
4,1781822,1659367871344,clicks,0
...,...,...,...,...
960804,1737908,1661723987073,clicks,12899776
960805,384045,1661723976974,clicks,12899777
960806,384045,1661723986800,clicks,12899777
960807,561560,1661723983611,clicks,12899778


2023-09-28 01:09:21,513 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 1 memory: 121 MB fds: 36>>
Traceback (most recent call last):
  File "/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/distributed/compatibility.py", line 161, in _run
    val = self.callback()
  File "/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/distributed/system_monitor.py", line 133, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/dkFePaHa/miniconda3/envs/tensorflow/lib/python3.9/site-packages/psutil/__init__.py", line 2119, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory
